In [36]:
import pandas as pd
import datetime
import os
import numpy as np

In [37]:
# Keep all stores for the 35+48+balance

In [38]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2018_Q2_Post/SOTF_Post'

In [43]:
store_list_48=pd.read_excel('/home/jian/Projects/Big_Lots/Analysis/2018_Q2_Post/SOTF_Post/BL SOTF 48 Stores_7.27 Zip Codes FINAL Kayla 20180712.xlsx',
                         dtype=str,sheetname="zips_by_store")
store_set_48=set(store_list_48['location_id'].unique().tolist())

In [45]:
store_list_35=pd.read_excel('/home/jian/Projects/Big_Lots/Analysis/2018_Q2_Post/SOTF_Post/SOTF Wave 1 Store IDs_8.20.xlsx',
                         dtype=str,sheetname="Stores_6_8")
store_set_35=set(store_list_35['SOTF'].unique().tolist())

In [47]:
store_set=set(list(store_set_48)+list(store_set_35))
len(store_set)

83

In [41]:
store_wide_sales=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2018_Q2_Post/SOTF_Post/wide_sales_date2018-09-01.xlsx",
                               dtype=str,sheetname="sales")

In [42]:
store_wide_trans=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2018_Q2_Post/SOTF_Post/wide_sales_date2018-09-01.xlsx",
                               dtype=str,sheetname="transactions")

In [48]:
First_week_2018=datetime.date(2018,7,7)
First_week_2017=datetime.date(2017,7,8)

weeks_2018=[str(First_week_2018+datetime.timedelta(days=i*7)) for i in range(8)]
weeks_2017=[str(First_week_2017+datetime.timedelta(days=i*7)) for i in range(8)]

In [49]:
all_store_sales=store_wide_sales[['location_id']+weeks_2017+weeks_2018]
# SOTF_sales=SOTF_sales[SOTF_sales['location_id'].isin(store_set)]
all_store_trans=store_wide_trans[['location_id']+weeks_2017+weeks_2018]
# SOTF_trans=SOTF_trans[SOTF_trans['location_id'].isin(store_set)]


In [50]:
for col in all_store_trans.columns.tolist()[1:]:
    all_store_trans[col]=all_store_trans[col].astype(int)
    
for col in all_store_sales.columns.tolist()[1:]:
    all_store_sales[col]=all_store_sales[col].apply(lambda x: np.round(float(x),2))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


# Rewards

In [9]:
start_2017=datetime.datetime.strptime(weeks_2017[0],"%Y-%m-%d").date()-datetime.timedelta(days=6)
start_2018=datetime.datetime.strptime(weeks_2018[0],"%Y-%m-%d").date()-datetime.timedelta(days=6)
end_2017=datetime.datetime.strptime(weeks_2017[7],"%Y-%m-%d").date()
end_2018=datetime.datetime.strptime(weeks_2018[7],"%Y-%m-%d").date()

In [10]:
data_from_SP=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/From_Sp/combinedtransactions_0811.csv",
                        dtype=str,usecols=['customer_id_hashed','transaction_date','transaction_id','location_id','total_transaction_amt'])
data_from_SP['transaction_date']=data_from_SP['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
data_from_SP=data_from_SP[(data_from_SP['transaction_date']>=start_2017) & (data_from_SP['transaction_date']<=end_2017) |\
                         (data_from_SP['transaction_date']>=start_2018) & (data_from_SP['transaction_date']<=end_2018)]
data_from_SP=data_from_SP.drop_duplicates()
data_from_SP['total_transaction_amt']=data_from_SP['total_transaction_amt'].astype(float)
data_from_SP.head(2)

,customer_id_hashed,transaction_date,transaction_id,location_id,total_transaction_amt
35777307,981ac89d974c5816e213220ff66499c0e9c3c020d35d00...,2018-07-02,9894,383,34.75
35777308,0c501ea9a5a4c619732b89599c93a23ea19d9c81b3a314...,2018-07-01,8690,5340,2.0


In [17]:
data_aug_28=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20180828-131144-584.txt",
                  dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','total_transaction_amt'])
data_aug_28=data_aug_28.drop_duplicates()
data_aug_28['transaction_dt']=data_aug_28['transaction_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
data_aug_28=data_aug_28.rename(columns={"transaction_dt":"transaction_date"})
data_aug_28['total_transaction_amt']=data_aug_28['total_transaction_amt'].astype(float)
data_aug_28.head(2)

,customer_id_hashed,transaction_date,transaction_id,location_id,total_transaction_amt
0,bf424b606c019216eb946c925572e5a9d63c94ea050dd7...,2018-08-24,8849,1267,43.65
1,9ba14b6eacc750af161a486c431b1adc39daf8a47c2007...,2018-08-12,4830,4341,6.60


In [19]:
rewards_data=data_from_SP.append(data_aug_28)
del data_from_SP
del data_aug_28
rewards_data=rewards_data.drop_duplicates()
del rewards_data['transaction_id']

In [27]:
def week_end_func(x):
    if x.weekday()==6:
        week=x+datetime.timedelta(days=6)
    else:
        diff=x.weekday()
        week=x+datetime.timedelta(days=5-diff)
    return week
        

In [28]:
rewards_data['week_end_date']=rewards_data['transaction_date'].apply(lambda x: week_end_func(x))


In [29]:
rewards_by_week_sales=rewards_data.groupby(['location_id','week_end_date'])['total_transaction_amt'].sum().to_frame().reset_index()
rewards_by_week_trans=rewards_data.groupby(['location_id','week_end_date'])['total_transaction_amt'].count().to_frame().reset_index()


In [54]:
all_store_sales.shape

(1513, 17)

# Combine

In [62]:
output=all_store_sales[['location_id']]


In [63]:
output_store_total_sales=pd.merge(output,all_store_sales,on="location_id",how="left")
output_store_total_trans=pd.merge(output,all_store_trans,on="location_id",how="left")

In [57]:
rewards_by_week_trans=rewards_by_week_trans.pivot(index="location_id",columns="week_end_date",values="total_transaction_amt").reset_index()
rewards_by_week_sales=rewards_by_week_sales.pivot(index="location_id",columns="week_end_date",values="total_transaction_amt").reset_index()


In [64]:
output_store_rewards_sales=pd.merge(output,rewards_by_week_sales,on="location_id",how="left")
output_store_rewards_trans=pd.merge(output,rewards_by_week_trans,on="location_id",how="left")

In [65]:
output_store_rewards_sales=output_store_rewards_sales.fillna(0)
output_store_rewards_trans=output_store_rewards_trans.fillna(0)

In [66]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2018_Q2_Post/SOTF_Post/"+"BL_48_35_stores_opened_20180727_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

In [67]:
output_store_total_sales.to_excel(writer,"store_total_sales",index=False)
output_store_total_trans.to_excel(writer,"store_total_trans",index=False)
output_store_rewards_sales.to_excel(writer,"store_rewards_sales",index=False)
output_store_rewards_trans.to_excel(writer,"store_rewards_trans",index=False)
writer.save()